In [13]:
from sch_viewer.parser import *
parser=tNavigatorModelParser()
model=parser.build_model(r'I:\proxy\test_model\NETWORK_DEMO.DATA')

nodeprop = model.find_keywords(keyword='NODEPROP')[0]
nodeprop.set_body_text(nodeprop.get_body_text().replace("'P' 22", "'P' 100"))

kw = model.find_keywords(keyword='NETCOMPA')[0]
netcompa_text = kw.get_body_text()
body_text=netcompa_text.replace("'C-", "--'C-").replace("100", "40")
kw.set_body_text(body_text)

model.save_as('proxy')

In [26]:
from sch_viewer.parser import *
parser=tNavigatorModelParser()
model=parser.build_model(r'C:\WORK\СППР ЦУД\tnav-for-proxy\test_model\proxy.DATA')

# 8 базовых вариантов
evt={
    0 : ['W5', 'W5-1', 'W5-2', 'W8', 'W9'], 
    1 : ['W11', 'W12', 'W13', 'W21', 'W22', 'W23', 'W41'], 
    2 : ['W50', 'W51', 'W52', 'W81', 'W82', 'W91', 'W92'], 
}

n=len(evt)
for v in range(2**n):
    bitstr=format(v, '0{}b'.format(n))
    wconprod = model.find_keywords(keyword="WCONPROD")[0]
    for i, s in enumerate(bitstr):
        if s=='0':
            for index, line in enumerate(wconprod.body):
                for well in evt[i]:
                    if line.startswith(well) or line.startswith(f"'{well}'"):
                        wconprod.body[index]=line.replace('OPEN', 'SHUT')
    model.save_as(f'{bitstr}-xxxx-xxxx-xxxx')
    model.return_to_original()

In [4]:
from sch_viewer.parser import *
from random import randint

evt=[['W5', 'W5-1', 'W5-2', 'W8', 'W9'], ['W11', 'W12', 'W13', 'W21', 'W22', 'W23', 'W41'], ['W50', 'W51', 'W52', 'W81', 'W82', 'W91', 'W92']]

p = tNavigatorModelParser()
model = p.build_model(r'C:\WORK\СППР ЦУД\tnav-for-proxy\test_model\proxy.DATA')

# количество элементов в выборке
minyear=min(model.schedule_data).year+3
# maxyear=max(model.schedule_data).year
maxyear=2026

modelvariants=[]
n=len(evt)
for v in range(2**n):
    bitstr=format(v, '0{}b'.format(n)) 
    for y1 in range(minyear, maxyear, 4) if bitstr[0]=='1' else ['xxxx']:
        for y2 in range(minyear, maxyear, 4) if bitstr[1]=='1' else ['xxxx']:
            for y3 in range(minyear, maxyear, 4) if bitstr[2]=='1' else ['xxxx']:
                name=f"{bitstr}-{y1}-{y2}-{y3}"
                modelvariants.append(name)

wconprod = model.delete_keywords(datetime(2012,3,1), keyword="WCONPROD")[0]
body = wconprod.body.copy()

# выключаем скважины в первоначальный период
kw = tNavigatorKeyword(wconprod.name)
kw.body = body.copy()
for index, line in enumerate(kw.body):
    for i in range(len(evt)):
        for well in evt[i]:
            if line.startswith(well) or line.startswith(f"'{well}'"):
                kw.body[index]=line.replace('OPEN', 'SHUT')
model.add_keyword(datetime(2012,3,1), kw)
        
for name in modelvariants:
    v = name.split('-')
    d = { 
        'Status' : [int(x) for x in v[0]], 
        'Year' : v[1:],
        'Event' : evt }    
    
    for i, b in enumerate(d['Status']):
        if b==1:
            kw = tNavigatorKeyword(wconprod.name)
            kw.add_line(wconprod.name)
            for index, line in enumerate(body):
                for well in d['Event'][i]:
                    if line.startswith(well) or line.startswith(f"'{well}'"):
                        kw.add_line(line)
            kw.add_line('/')
            model.add_keyword(datetime(int(d['Year'][i]), 1, 1), kw)
    model.save_as(name)
    for year in d['Year']:
        if year!='xxxx' and len(model.find_keywords(datetime(int(year),1,1), keyword=wconprod.name))>0:
            model.delete_keywords(datetime(int(year),1,1), keyword=wconprod.name)